> **This notebook uses a langchain `create_sql_agent` to query a SQL database.**
> - **Thought:** Use this agent for various purposes: querying data, generating reports, authoring documents, and maybe updating data
> - **Observations:**
>   - LangChain `SQLDatabase` can only work with the default schema, until [this issue](https://github.com/langchain-ai/langchain/issues/3036) is resolved.
> - Sample Data: [Northwind Traders](https://www.dofactory.com/sql/sample-database), bacause AdventureWorks uses a `SalesLT` schema

## Environment Setup

In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]

SQL_DB_USER = os.environ["SQL_DB_USER"]
SQL_DB_PASSWORD = os.environ["SQL_DB_PASSWORD"]
SQL_DB_SERVER_NAME = os.environ["SQL_DB_SERVER_NAME"]
SQL_DB_NAME = os.environ["SQL_DB_NAME"]
SQL_CONNECTIONSTRING_FORMAT = os.environ["SQL_CONNECTIONSTRING_FORMAT"]

In [2]:
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

llm = AzureOpenAI(
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    #temperature=0.3,
    verbose=True
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION,
    #temperature=0.3,
    verbose=True
)

## Code

In [3]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

connection_string = SQL_CONNECTIONSTRING_FORMAT.format(
    database_user=SQL_DB_USER,
    database_password=SQL_DB_PASSWORD,
    database_server=SQL_DB_SERVER_NAME,
    database_db=SQL_DB_NAME)

db = SQLDatabase.from_uri(connection_string)
toolkit = SQLDatabaseToolkit(db=db, llm=chat)

In [4]:
from langchain.agents import create_sql_agent

sqlagent = create_sql_agent(
    llm=chat,
    toolkit=toolkit,
    verbose=True
)

In [5]:
sqlagent.run("list top 5 customers by highest order amount")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Order, OrderItem, Product, Supplier
Thought:The tables that seem relevant to the question are Customer and Order. I should query the schema of these tables.
Action: sql_db_schema
Action Input: "Customer, Order"
Observation: 
CREATE TABLE [Customer] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[FirstName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[LastName] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[City] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Country] NVARCHAR(40) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Phone] NVARCHAR(20) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_CUSTOMER] PRIMARY KEY ([Id])
)

/*
3 rows from Customer table:
Id	FirstName	LastName	City	Country	Phone
1	Maria	Anders	Berlin	Germany	030-0074321
2	Ana	Trujillo	México D.F.	Mexico	(5) 555-4729
3	Antonio	Moreno	México D.F.	Mexico	(5) 555-3932

'Horst Kloss, Jose Pavarotti, Mario Pontes, Lúcia Carvalho, Jean Fresnière'

In [6]:
sqlagent.run("Get the Product with the top orders by Amount, and then group by Country.")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Customer, Order, OrderItem, Product, Supplier
Thought:I can query the Order and OrderItem tables to get the top orders by amount. Then I can join the Product table to get the product information. I should also group the results by country.
Action: sql_db_schema
Action Input: "Order, OrderItem, Product"
Observation: 
CREATE TABLE [OrderItem] (
	[Id] INTEGER NOT NULL IDENTITY(1,1), 
	[OrderId] INTEGER NOT NULL, 
	[ProductId] INTEGER NOT NULL, 
	[UnitPrice] DECIMAL(12, 2) NOT NULL DEFAULT ((0)), 
	[Quantity] INTEGER NOT NULL DEFAULT ((1)), 
	CONSTRAINT [PK_ORDERITEM] PRIMARY KEY ([Id]), 
	CONSTRAINT [FK_ORDERITE_REFERENCE_ORDER] FOREIGN KEY([OrderId]) REFERENCES [Order] ([Id]), 
	CONSTRAINT [FK_ORDERITE_REFERENCE_PRODUCT] FOREIGN KEY([ProductId]) REFERENCES [Product] ([Id])
)

/*
3 rows from OrderItem table:
Id	OrderId	ProductId	UnitPrice	Quantity
1	1	11	14.00	12
2	1	42	9.80	10
3	1	72	34.80	5


InvalidRequestError: This model's maximum context length is 16384 tokens. However, your messages resulted in 19274 tokens. Please reduce the length of the messages.